In [3]:
import pandas as pd 

In [4]:
data = pd.read_csv(r"C:\Users\rajse\OneDrive\Desktop\Ex_Files_NLP_Python_ML_EssT/SMSSpamCollection.tsv",sep = '\t', header = None)
data.columns = ['labels', 'body_text']
data.head()

,labels,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [5]:
import nltk
import pandas as pd 
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import string 
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
ps=nltk.PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajse\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [6]:
# length of character = len(string)- count of white-spaces
data['body_len']=data['body_text'].apply(lambda x :len(x)-x.count(" "))


In [7]:

#Creating a function to fin dout the % of punctuation present in the text
def count_punct(text):
  count = sum([1 for char in text if char in string.punctuation])
  return round(count/(len(text) - text.count(" ")), 3)*100
    
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

data.head(2)

,labels,body_text,body_len,punct%
0,ham,I've been searching for the right words to tha...,160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,128,4.7


In [8]:
def clean_text(text):
  text = "".join([word.lower() for word in text if word not in string.punctuation])#Punctuation removal
  tokens = re.split('\W+', text) #tokenize
  text = [ps.stem(word) for word in tokens if word not in stopwords] # Stopwords removal
  return text

In [9]:
tfidf_vect  = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
#count_vect  = CountVectorizer(analyzer=clean_text)
#X_tfidf = count_vect.fit_transform(data['body_text'])

In [10]:
X_features = pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis= 1)

In [13]:
X_features.head(2)

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


 Explore  Gradient-Boosting Classifier Attributes & Hyperparameters

In [14]:
from  sklearn.ensemble import GradientBoostingClassifier
print(dir(GradientBoostingClassifier))

['_SUPPORTED_LOSS', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_initialized', '_check_n_features', '_check_params', '_clear_state', '_compute_partial_dependence_recursion', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_more_tags', '_raw_predict', '_raw_predict_init', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_data', '_validate_estimator', '_validate_y', '_warn_mae_for_criterion', 'apply

In [15]:
from sklearn.metrics import precision_recall_fscore_support as score 
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X_features,data['labels'],test_size=0.2)


In [23]:
def train_GB(est, max_depth, lr):
        gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
        gb_model = gb.fit(X_train, y_train)
        y_pred = gb_model.predict(X_test)
        precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
        print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                                                est, max_depth, lr, round(precision, 3), round(recall, 3),
                                                                                round((y_pred==y_test).sum()/len(y_pred), 3)))

In [26]:
for n_est in [50]:
       for max_depth in [3, 7, 11, 15]:
            for lr in [0.01, 0.1, 1]:
                train_GB(n_est, max_depth, lr)


C:\Users\rajse\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.87
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.938 / Recall: 0.731 / Accuracy: 0.959
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.886 / Recall: 0.807 / Accuracy: 0.961
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.871
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.931 / Recall: 0.834 / Accuracy: 0.97
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.904 / Recall: 0.841 / Accuracy: 0.968
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.014 / Accuracy: 0.872
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.916 / Recall: 0.828 / Accuracy: 0.968
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.922 / Recall: 0.821 / Accuracy: 0.968
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.871
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.938 / Recall: 0.828 / Accuracy: 0.97
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.909 / Recall: 0.828 / Accurac